In [1]:
# Dataset : Titanic
# Author : kcw
# Date : 20231125
# Tags : Data visualization, Python

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [85]:
path = './dataset/'
dataset_path = path + 'train.csv'
df = pd.read_csv(dataset_path)
df.sample(10) # 🕊️ head는 순서대로 출력하기 때문에 중요한 특징을 누락할 수 있어요.

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
184,185,1,3,"Kink-Heilmann, Miss. Luise Gretchen",female,4.0,0,2,315153,22.0250,NaN,S
174,175,0,1,"Smith, Mr. James Clinch",male,56.0,0,0,17764,30.6958,A7,C
77,78,0,3,"Moutal, Mr. Rahamin Haim",male,NaN,0,0,374746,8.0500,NaN,S
590,591,0,3,"Rintamaki, Mr. Matti",male,35.0,0,0,STON/O 2. 3101273,7.1250,NaN,S
207,208,1,3,"Albimona, Mr. Nassef Cassem",male,26.0,0,0,2699,18.7875,NaN,C
798,799,0,3,"Ibrahim Shawah, Mr. Yousseff",male,30.0,0,0,2685,7.2292,NaN,C
306,307,1,1,"Fleming, Miss. Margaret",female,NaN,0,0,17421,110.8833,NaN,C
20,21,0,2,"Fynney, Mr. Joseph J",male,35.0,0,0,239865,26.0000,NaN,S
768,769,0,3,"Moran, Mr. Daniel J",male,NaN,1,0,371110,24.1500,NaN,Q
761,762,0,3,"Nirva, Mr. Iisakki Antino Aijo",male,41.0,0,0,SOTON/O2 3101272,7.1250,NaN,S


In [134]:
#  사전작업 - 각 feature의 형태 파악

result = "" # 🕊️ EDA 결과를 차곡차곡 저장해서 마지막에 출력해주려해요.

info_result = 'info ' + '='*100 + '\n' # 🕊️ result 변수는 최종 출력물만 넣기 위해 다른 변수를 선언했어요.

info_result += "columns: "
for col in df.columns.values:
    info_result += f"{col}, "
info_result = info_result[:-2] + '\n'

info_result += f"양적변수 > 연속형 : Fare\n"
var_num_con = ['Fare']
info_result += f"양적변수 > 이산형 : Age, SibSp, Parch\n"
var_num_dis = ['Age', 'SibSp', 'Parch']
info_result += f"질적변수 > 명목형 : Sex, PassengerId, Survived, Ticket, Cabin, Embarked, Name\n"
var_cat_non = ['Sex', 'PassengerId', 'Survived', 'Ticket', 'Cabin', 'Embarked', 'Name']
info_result += f"질적변수 > 순서형 : Pclass\n"
var_cat_ord = ['Pclass']
for col in ['Sex', 'PassengerId', 'Survived', 'Ticket', 'Cabin', 'Embarked', 'Name', 'Pclass']:
    df[col] = df[col].astype('category')
info_result += f"- 'Age'에 177개의 누락된 값이 있습니다.\n"
info_result += f"- 'Cabin'은 687개의 누락된 값이 있습니다.\n"
info_result += f"- 'Embarked'에 2개의 누락된 값이 있습니다.\n"
info_result += f"- 질적변수의 자료형을 모두 Category로 변경해주었습니다.\n"

print(info_result)

info ====================================================================================================
columns: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Family
양적변수 > 연속형 : Fare
양적변수 > 이산형 : Age, SibSp, Parch
질적변수 > 명목형 : Sex, PassengerId, Survived, Ticket, Cabin, Embarked, Name
질적변수 > 순서형 : Pclass
- 'Age'에 177개의 누락된 값이 있습니다.
- 'Cabin'은 687개의 누락된 값이 있습니다.
- 'Embarked'에 2개의 누락된 값이 있습니다.
- 질적변수의 자료형을 모두 Category로 변경해주었습니다.



In [118]:
# 전처리 - 파생 변수의 고려

var_result = 'variable ' + '='*100 + '\n'

df['Family'] = df.apply(lambda x: int(x.SibSp>0 and x.Parch>0), axis=1)
var_result += "파생변수 Family : 가족이 동승하였다면 1, 아니면 0으로 두었습니다.\n"
var_result += "추가 파생변수는 task에 맞춰, 그리고 시각화 결과를 보고 추가하고자 합니다.\n"

print(var_result)

variable ====================================================================================================
파생변수 Family : 가족이 동승하였다면 1, 아니면 0으로 두었습니다.
추가 파생변수는 task에 맞춰, 그리고 시각화 결과를 보고 추가하고자 합니다.



In [132]:
# 전처리 - 결측치 처리

missing_result = 'missing ' + '='*100 + '\n'

missing_result += f"'Age' 변수의 경우 목적 변수인 'Survived'에 영향을 미칠 것으로 기대하기 때문에 대체값을 사용하기보다 관련 분석 시 NaN값을 제외하는 방향으로 설계하였습니다.\n"
missing_result += f"'Cabin' 변수 역시 마찬가지의 이유로 Unknown 카테고리를 새롭게 추가하여 결측값을 처리하였습니다. \n"
df.Cabin = df.Cabin.cat.add_categories('Unknown')
df.Cabin = df.Cabin.fillna('Unknown')
missing_result += f"'Embarked' 변수의 경우 목적 변수인 'Survived'에 별 영향을 미치지 않을 것으로 기대하기 때문에 시각화 후 변수의 제거를 고려해보려 합니다."

print(missing_result)

missing ====================================================================================================
'Age' 변수의 경우 목적 변수인 'Survived'에 영향을 미칠 것으로 기대하기 때문에 대체값을 사용하기보다 관련 분석 시 NaN값을 제외하는 방향으로 설계하였습니다.
'Cabin' 변수 역시 마찬가지의 이유로 Unknown 카테고리를 새롭게 추가하여 결측값을 처리하였습니다. 
'Embarked' 변수의 경우 목적 변수인 'Survived'에 별 영향을 미치지 않을 것으로 기대하기 때문에 시각화 후 변수의 제거를 고려해보려 합니다.


In [135]:
# print result

result = f"{info_result}\n{var_result}\n{missing_result}"

print(result)

info ====================================================================================================
columns: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fare, Cabin, Embarked, Family
양적변수 > 연속형 : Fare
양적변수 > 이산형 : Age, SibSp, Parch
질적변수 > 명목형 : Sex, PassengerId, Survived, Ticket, Cabin, Embarked, Name
질적변수 > 순서형 : Pclass
- 'Age'에 177개의 누락된 값이 있습니다.
- 'Cabin'은 687개의 누락된 값이 있습니다.
- 'Embarked'에 2개의 누락된 값이 있습니다.
- 질적변수의 자료형을 모두 Category로 변경해주었습니다.

variable ====================================================================================================
파생변수 Family : 가족이 동승하였다면 1, 아니면 0으로 두었습니다.
추가 파생변수는 task에 맞춰, 그리고 시각화 결과를 보고 추가하고자 합니다.

missing ====================================================================================================
'Age' 변수의 경우 목적 변수인 'Survived'에 영향을 미칠 것으로 기대하기 때문에 대체값을 사용하기보다 관련 분석 시 NaN값을 제외하는 방향으로 설계하였습니다.
'Cabin' 변수 역시 마찬가지의 이유로 Unknown 카테고리를 새롭게 추가하여 결측값을 처리하였습니다. 
'Embarked' 변수의 경우 목적 변수인 'Survived'에 별 영향을 미치지 않을 것으로

In [140]:
# publish dataset and EDA result

import pickle

with open('./dataset/train_after_eda.pkl', 'wb') as f:
    pickle.dump(df, f)

with open('./dataset/train_after_eda.pkl', 'rb') as f:
    pickle.load(f).info()

with open('./dataset/eda_result.txt', 'w') as f:
    f.write(result)

with open('./dataset/eda_result.txt', 'r') as f:
    print(f.read())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   PassengerId  891 non-null    category
 1   Survived     891 non-null    category
 2   Pclass       891 non-null    category
 3   Name         891 non-null    category
 4   Sex          891 non-null    category
 5   Age          714 non-null    float64 
 6   SibSp        891 non-null    int64   
 7   Parch        891 non-null    int64   
 8   Ticket       891 non-null    category
 9   Fare         891 non-null    float64 
 10  Cabin        891 non-null    category
 11  Embarked     889 non-null    category
 12  Family       891 non-null    int64   
dtypes: category(8), float64(2), int64(3)
memory usage: 65.8 KB
info ====================================================================================================
columns: PassengerId, Survived, Pclass, Name, Sex, Age, SibSp, Parch, Ticket, Fa